# exploration notebook
craig has provided two `kml` files, one delimiting the area of interest, another listing a number of wells within the area. we are interested in fetching planet.com data within the area of interest, so we will need to parse that file. 

`kml` files are a peculiar file format. tried accessing and parsing with `fastkml`, but eventually gave up due to lack of consistent documentation,  switched to `elementree`. even that did not go smoothly but we can now access all the well coordinates.

these 

In [1]:
import xml.etree.ElementTree as ET

In [2]:
filepath_aoi  = 'data/pabal-AOI.kml'
filepath_well = 'data/Pabal-Wells.kml'

## read in wells

In [21]:
import xml.etree.ElementTree as ET
tree = ET.parse(filepath_well)
root = tree.getroot()
# print(root.attrib, root.text, root)
docs = root.findall('{http://www.opengis.net/kml/2.2}kml')
docs
# placemarks = root.findall('Placemark')
# placemarks
# huh?

[]

In [40]:
tree = ET.parse(filepath_well)
root = tree.getroot()
well_coordinates = []
for elem in root:
    for subelem in elem:
        if subelem.tag == '{http://www.opengis.net/kml/2.2}Placemark':
            for subsubelem in subelem:
                if subsubelem.tag == '{http://www.opengis.net/kml/2.2}Point':
#                     print(subsubelem.tag, subsubelem.attrib, subsubelem.items())
                    for subsubsubelem in subsubelem:
#                         print('tag =', subsubsubelem.tag, 'attrib=', subsubsubelem.attrib, subsubsubelem.items(), subsubsubelem.text )
                        coords = [float(num_str) for num_str in subsubsubelem.text.split(',')]
                        well_coordinates.append([coords[0], coords[1]]) # lat lon only. discard z coordinate.
print('collected', len(well_coordinates), 'well coordinates')
# well_coordinates

collected 257 well coordinates


In [41]:
tree = ET.parse(filepath_aoi)
root = tree.getroot()
aoi_polygon_pts = []
for elem in root:
    for subelem in elem:
        for subsubelem in subelem:
#             print(subsubelem, type(subsubelem), subsubelem.tag, subsubelem.attrib, subsubelem.items())
            if subsubelem.tag == '{http://www.opengis.net/kml/2.2}name':
                name = subsubelem.text
                print('aoi name:', name)
            elif subsubelem.tag == '{http://www.opengis.net/kml/2.2}Placemark':
                for subsubsubelem in subsubelem:
                    if subsubsubelem.tag == '{http://www.opengis.net/kml/2.2}Style':
                        for style in subsubsubelem:
                            print(style, type(style))
                    elif subsubsubelem.tag == '{http://www.opengis.net/kml/2.2}Polygon':
                        for boundary in subsubsubelem:
                            for linear_ring in boundary:
                                for point in linear_ring:
                                    corners = point.text.split(' ')
                                    for coords in corners:
#                                         print(coords)
                                        aoi_polygon_pts.append([float(num_str) for num_str in coords.split(',')])

print('collected', len(aoi_polygon_pts), 'aoi polygon point coordinates')
# aoi_polygon_pts

aoi name: pabal
<Element '{http://www.opengis.net/kml/2.2}LineStyle' at 0x1089ef2c0> <class 'xml.etree.ElementTree.Element'>
<Element '{http://www.opengis.net/kml/2.2}PolyStyle' at 0x1089ef040> <class 'xml.etree.ElementTree.Element'>
collected 765 aoi polygon point coordinates


In [23]:
k_aoi = kml.KML()
k_aoi.from_string(aoi)

In [28]:
with open(filepath_well, 'rt') as wellfile:
    wells=wellfile.read().encode('utf-8')

k_wells = kml.KML()
k_wells.from_string(wells)

In [29]:
# inspection
print('parsed object has', k_aoi, type(k_aoi))
features_aoi = list(k_aoi.features())
print('there are', len(features_aoi), 'features in aoi file')
features_2_aoi = list(features_aoi[0].features())
print('the first feature has', len(features_2_aoi), 'subfeatures of type', type(features_2_aoi[0]))
print('description of first subfeature', features_2_aoi[0].description)
print('name of first subfeature', features_2_aoi[0].name)
print('dir of subfeature', dir(features_2_aoi[0]))

parsed object has <fastkml.kml.KML object at 0x1117bb8e0> <class 'fastkml.kml.KML'>
there are 1 features in aoi file
the first feature has 1 subfeatures of type <class 'fastkml.kml.Folder'>
description of first subfeature None
name of first subfeature pabal
dir of subfeature ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_address', '_atom_author', '_atom_link', '_features', '_phoneNumber', '_snippet', '_styleUrl', '_styles', '_time_span', '_time_stamp', 'address', 'append', 'append_style', 'author', 'begin', 'description', 'end', 'etree_element', 'extended_data', 'features', 'from_element', 'from_string', 'id', 'isopen', 'link', 'name', 'ns', 'phoneNumber', 'snippet', 'styleUrl', 

In [30]:
# inspection
print('parsed object has', k_wells, type(k_wells))
features_wells = list(k_wells.features())
print('there are', len(features_wells), 'features in wells file')
features_2_wells = list(features_wells[0].features())
print('the first feature has', len(features_2_wells), 'subfeatures of type', type(features_2_wells[0]))
print('description of first subfeature', features_2_wells[0].description)
print('name of first subfeature', features_2_wells[0].name)
print('dir of subfeature', dir(features_2_wells[0]))

parsed object has <fastkml.kml.KML object at 0x11187f9a0> <class 'fastkml.kml.KML'>
there are 1 features in wells file
the first feature has 257 subfeatures of type <class 'fastkml.kml.Placemark'>
description of first subfeature <!-- desc_gen:start -->
<font color="black"><table width="100%"><tr><td width="100%" align="center">
<!-- desc_user:start -->
<br /><table style="width:100%; table-layout:fixed;"><tr><td style="width:150px; align:right; valign:center;"><small><b>system:index</b></small></td><td style="align:center; valign:center; word-wrap:break-word;">0</td></tr>
</table>
<!-- desc_user:end -->
</td></tr><tr><td><table width="100%"></table></td></tr></table></font>
<!-- desc_gen:end -->
name of first subfeature None
dir of subfeature ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__reduc

In [42]:
for feature in features_2_wells:
    point = feature.geometry
    print(type(point), point)
    geometry = feature.geometry
    if geometry:
        print('feature geometry =', geometry, dir(geometry))


NameError: name 'features_2_wells' is not defined

In [31]:
from planet import api
import geojson
path_to_file = 'data/pabal.geojson'

with open(path_to_file) as f:
    gj = geojson.load(f)
features = gj['features'][0]
gj['features'][0]['geometry'] 

{"coordinates": [[[74.063134, 18.858184], [74.064011, 18.858094], [74.064578, 18.858035], [74.064721, 18.858015], [74.065123, 18.857957], [74.06524, 18.857961], [74.065657, 18.857982], [74.066215, 18.857936], [74.066259, 18.857933], [74.06638, 18.857904], [74.066133, 18.857549], [74.065824, 18.857074], [74.065389, 18.856524], [74.065024, 18.856058], [74.064882, 18.855463], [74.064879, 18.855397], [74.064919, 18.85536], [74.064919, 18.85536], [74.064919, 18.855358], [74.064927, 18.855356], [74.065756, 18.855188], [74.066517, 18.855065], [74.067516, 18.854865], [74.067515, 18.854858], [74.067503, 18.854634], [74.067503, 18.854626], [74.067529, 18.854541], [74.067609, 18.854387], [74.067616, 18.854372], [74.067584, 18.854275], [74.067455, 18.85416], [74.067368, 18.854082], [74.067357, 18.854017], [74.067407, 18.853669], [74.067496, 18.853044], [74.067513, 18.852927], [74.06755, 18.852788], [74.067582, 18.852668], [74.067617, 18.852537], [74.067676, 18.852512], [74.067696, 18.852503], [74.